In [71]:
import pandas as pd
import numpy as np

In [72]:
books = pd.read_csv('data/book_data.csv')

In [73]:
books.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [74]:
#cleaning book format
books.book_format = books.book_format.str.strip().str.lower()

In [75]:
#cleaning author names
books.book_authors = list(books.book_authors.str.split('|'))
#Creating a new row for every listed genre
authors = books.apply(lambda x: pd.Series(x.book_authors),axis=1).stack().reset_index(level=1, drop=True)
authors.name = 'author'
books = books.drop('book_authors', axis=1).join(authors).drop_duplicates()

In [76]:
len(books)

73053

In [77]:
#cleaning descriptions


In [78]:
#cleaning book pages

In [79]:
#cleaning genre
books.genres = list(books.genres.str.split('|'))
#Creating a new row for every listed genre
genres = books.apply(lambda x: pd.Series(x.genres),axis=1).stack().reset_index(level=1, drop=True)
genres.name = 'genre'
books = books.drop('genres', axis=1).join(genres).drop_duplicates()

In [80]:
books.head()

,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,image_url,author,genre
0,Winning will make you famous. Losing means cer...,NaN,hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,Young Adult
0,Winning will make you famous. Losing means cer...,NaN,hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,Fiction
0,Winning will make you famous. Losing means cer...,NaN,hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,Science Fiction
0,Winning will make you famous. Losing means cer...,NaN,hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,Dystopia
0,Winning will make you famous. Losing means cer...,NaN,hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,Suzanne Collins,Fantasy


In [85]:
#selecting columns
books = books[['book_title','author','genre' ,'book_rating', 'book_pages','book_format', 'book_desc']]
len(books)

360186

In [84]:
#write to csv
books.to_csv('data/clean_books.csv')